# Scalper's Statistics:  PlayStation 5
### By Lee Thomas, Taylor Johnson, Alex Peña, and Paul Leonard
#### December 2020

##### Import related libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from datetime import datetime


##### Import data sets

In [ ]:
df_ebay = pd.read_csv("./data/ps4_scalped.csv")
df_msrp = pd.read_csv("./data/ps4_msrp.csv")
df_real_ebay = pd.read_csv("./fake-data/ebay_data.csv")

In [ ]:
df_ebay.head(5)
df_msrp.head(5)

##### Combine datafames

In [ ]:
# consider combining each of the dataframes into one large one called df_ps5
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

# dfs_to_combine = [df_ebay,df_msrp]
# df_combo = pd.concat(dfs_to_combine)
# df_combo.head(5)

df_combo = df_msrp
df_combo["ebay"] = df_ebay["price"]
df_combo["price_delta"] = df_combo.ebay - df_combo.price
df_combo["percent_markup"] = (df_combo.ebay / df_combo.price * 100 - 100).round(2)

# NEED TO GET DATES TO DATETIME OBJECTS TO FIGURE OUT DAYS SINCE RELEASE
# df_combo["date_object"] = datetime.strptime(df_combo["date"],'%m/%d/%y')
# df_combo["date_object"] = datetime.fromisoformat(str(df_combo["date"]))
# above did not work... need to use pandas datetools parser to get things sorted out; think it has to use pd.apply()... check out this page:
# https://stackoverflow.com/questions/18618288/how-do-i-convert-dates-into-iso-8601-datetime-format-in-a-pandas-dataframe

# AFTER TO A DATETIME OBJECT... USE MATH OF THIS SORT TO CALCULATE THE DURATION
df_combo["days_since_release"] = df_combo["date_object"] - df_combo["date_object"].min

df_combo.head(5)


## Time History Plot of PS5 Prices

In [ ]:
plt.subplots(figsize=(10,4))
plt.scatter(df_ebay["date"].values,df_ebay["price"].values)
plt.xticks(rotation='vertical')
plt.grid(True)

plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
g = sns.lineplot(x = df_ebay["date"].values,
    y = df_ebay["price"].values,
    color = 'r',
    ax = ax)
sns.lineplot(x = df_msrp["date"].values,
    y = df_msrp["price"].values,
    color = 'b',
    ax = ax)
ax2 = ax.twinx()
ax.legend(['eBay','MSRP'], facecolor='w')
ax.set(xlabel="Date",ylabel="Price")
plt.title("PlayStation 5 Price History")
plt.xticks(rotation='vertical')
plt.grid(True)
g.set_xticklabels(df_ebay["date"].values, rotation=45)

plt.show()


##  What is the percent markup being charged by scalpers on the PS5?

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.scatterplot(x = df_combo["date"].values,
    y = df_combo["percent_markup"].values,
    color = 'r',
    ax = ax)

ax.legend(['% markup'], facecolor='w')
ax.set(xlabel="Date",ylabel="% Markup")
plt.title("PlayStation 5 Third Party Markup History")
plt.xticks(rotation='vertical')
plt.grid(True)

plt.show()

## What if iPad Air 64GB (4th gen) at $599 is scalped at these rates?

In [ ]:
df_combo["ipad"] = ( df_combo["percent_markup"] / 100 + 1 ) * 599

fig, ax = plt.subplots(figsize=(10,4))
sns.lineplot(x = df_combo["date"].values,
    y = df_combo["ipad"].values,
    color = 'r',
    ax = ax)
ax.legend(['iPad'], facecolor='w')
ax.set(xlabel="Date",ylabel="Price")
plt.title("iPad Air (64GB 4th gen) Scalper Comparison")
plt.xticks(rotation='vertical')
plt.grid(True)


plt.show()

## What if MacBook Pro 13" ($1299) was scalped at these rates?

In [ ]:
df_combo["macbook"] = ( df_combo["percent_markup"] / 100 + 1 ) * 1299

fig, ax = plt.subplots(figsize=(12,4))
sns.lineplot(x = df_combo["date"].values,
    y = df_combo["macbook"].values,
    color = 'r',
    ax = ax)
ax.legend(['MacBook Pro 13"'], facecolor='w')
ax.set(xlabel="Date",ylabel="Price")
plt.title('MacBook Pro 13" Scalper Comparison')
plt.xticks(rotation=45)
plt.grid(True)


plt.show()

## What if this same markup was applied to chapstick (MSRP $1.99) or Dove Deep Moisture Nourishing Body Wash(MSRP $5.89)?

(https://www.target.com/p/dove-deep-moisture-nourishing-body-wash-for-dry-skin-22-fl-oz/-/A-11695274)

In [ ]:
df_combo["chapstick"] = ( df_combo["percent_markup"] / 100 + 1 )* 1.99
df_combo["bodywash"] = ( df_combo["percent_markup"] / 100 + 1 ) * 5.89

fig, ax = plt.subplots(figsize=(10,4))
sns.lineplot(x = df_combo["date"].values,
    y = df_combo["chapstick"].values,
    color = 'r',
    ax = ax)
sns.lineplot(x = df_combo["date"].values,
    y = df_combo["bodywash"].values,
    color = 'b',
    ax = ax)
ax.legend(['ChapStick','Dove Body Wash'], facecolor='w')
ax.set(xlabel="Date",ylabel="Price")
plt.title("Common Item Scalper Comparison")
plt.xticks(rotation='vertical')
plt.grid(True)

plt.show()

In [ ]:
df_combo.index.values
df_combo["percent_markup"].values

## Fitting the markup data with a non-linear regression

In [ ]:
# delete the next line once this is calculated at the generation of df_combo
df_combo["days_since_release"] = df_combo.index.values
df_combo.head(5)



In [ ]:
ps5regressionplot = sns.lmplot(x="days_since_release", y="percent_markup", data=df_combo, order=3,  ci=None, line_kws={'color': 'red'}, scatter_kws={"s": 80}, height=5, aspect=2)


ps5regressionplot.set(
    title="Regression Fit for PS5 Third Party Prices",
    xlim=(0,50),
    # xticks=[50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000],
    ylim=(0,250),
    yticks=[0,50,100,150,200,250],)
# ps5regressionplot.suptitle('bold figure suptitle', fontsize=14, fontweight='bold')

## In defining the regression, we knew we wanted to keep the order low, so we started generating regression with a quadratic and 3rd order regressions... However, in plotting and visually testing the results for fit, we saw that a higher order regression was needed.  (Otherwise... buy your PS5 now!!)

In [ ]:
ps5regressionplot = sns.lmplot(x="days_since_release", y="percent_markup", data=df_combo, order=3,  ci=None, line_kws={'color': 'red'}, scatter_kws={"s": 80}, height=5, aspect=2)


ps5regressionplot.set(
    title="Regression Fit for PS5 Third Party Prices",
    xlim=(0,1000),
    xticks=[50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000],
    ylim=(0,400),
    yticks=[0,50,100,150,200,250,300,350,400],)
# ps5regressionplot.suptitle('bold figure suptitle', fontsize=14, fontweight='bold')

## OK... seaborn will not give regression line equations... so let's try SKlearn with Support Vector Regression (SVR)

In [ ]:
X=df_combo["days_since_release"].values.reshape(-1,1)
y=df_combo["percent_markup"].values

svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_lin = SVR(kernel='linear', C=100, gamma='auto')
svr_poly = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1, coef0=1)

lw=2

svrs = [svr_rbf, svr_lin, svr_poly]
kernel_label = ['RBF', 'Linear', 'Polynomial']
model_color = ['m', 'c', 'g']

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(15,15), sharex=True)
for ix, svr in enumerate(svrs):
    axes[ix].plot(X, svr.fit(X, y).predict(X), color=model_color[ix], lw=lw, label='{} model'.format(kernel_label[ix]))
    axes[ix].scatter(X[svr.support_], y[svr.support_], facecolor="none", edgecolor=model_color[ix], s=50, label='{} support vectors'.format(kernel_label[ix]))    
    axes[ix].scatter(X[np.setdiff1d(np.arange(len(X)), svr.support_)], y[np.setdiff1d(np.arange(len(X)), svr.support_)], facecolor="none", edgecolor="k", s=50, label='other training data')
    axes[ix].legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=1, fancybox=True, shadow=True)    

fig.text(0.5, 0.04, 'Days Since Initial Release', ha='center', va='center')
fig.text(0.06, 0.5, 'Third Party Price', ha='center', va='center', rotation='vertical')
fig.suptitle("PS4 Third Party Prices for Days Since Initial Release\nSupport Vector Regression", fontsize=14)
plt.show()

## Hexbin plot with distributions:  shows heat map of sale price/volume density;  x-axis days; y-axis price

In [ ]:
orchestra = sns.violinplot(x=df_ebay["price"], inner="box", orient='')

In [ ]:
# Above is a representation of the entire data set as a violin plot. This shows the majority of sales happen between $400-$500, then $700-$900, with outliers from $200-$1600+.

In [ ]:
mean_ = df_ebay["price"].mean()
median_ = df_ebay["price"].median()
mode_ = df_ebay["price"].mode()

mean_

# Here we see that the mean of the scalped prices is $708.


In [ ]:
orchestra = sns.violinplot(x=df_real_ebay['Price'], inner='box')

# Monday Violin


In [ ]:
mean_ = df_real_ebay.mean()
mean_

In [ ]:
df_ebay_tues = pd.read_csv("./data/ebay_data_tuesday.csv")
orchestra = sns.violinplot(x=df_ebay_tues['Price'], inner='box')

# Tuesday Violin

In [ ]:
mean_ = df_ebay_tues.mean()
mean_

### Line Plots of the PS5 MSRP and Secondary Market with Sony Units Sold Count as Bar Chart

Search the linked site for "Or, we can combine a bar chart and a line chart with the same x-axis but different y-axis:" to get more information on how to make it.  The site is already listed in the README.

https://towardsdatascience.com/a-step-by-step-guide-for-creating-advanced-python-data-visualizations-with-seaborn-matplotlib-1579d6a1a7d0

### Violin Chart for each day PS5 has been released... grouping by day, height by max to min price, width of units sold on ebay

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.scatterplot(x = df_ebay["date"].values,
    y = df_ebay["price"].values,
    color = 'r',
    ax = ax)
sns.scatterplot(x = df_msrp["date"].values,
    y = df_msrp["price"].values,
    color = 'b',
    ax = ax)

# ax2 = ax.twinx()
# sns.scatterplot(x = df_combo["date"].values,
#     y = df_combo["percent_markup"].values,
#     color = "orange",
#     ax = ax2)

ax = df_combo.plot(secondary_y=["percent_markup"])

ax.legend(['eBay','MSRP','% markup'], facecolor='w')
# ax2.legend(['% markup'], facecolor='w')
# ax2.set(ylim(0,500))
ax.set(xlabel="Date",ylabel="Price")
# ax.right_ax.set_ylabel('% Markup')
plt.title("PlayStation 5 Price History")
plt.show()